## **Import Packages**

In [ ]:
# Import Packages
from bs4 import BeautifulSoup
from datetime import datetime 
import requests   
import pandas as pd 
import numpy as np
from tqdm import trange
pd.set_option('display.max_rows',1000)
import time
from google.cloud import bigquery

## **Initialize Client Object**

In [ ]:
# initialize client object
client = bigquery.Client()

## **Scrape Real Estate Listings in Nairobi & Mombasa, Kenya**

### **a. Apartments for Rent in Nairobi**

In [ ]:
# Initialize DataFrame outside the loop
bigdata = pd.DataFrame()

for page in trange(1,101):
    counties = ['nairobi']
    furnished = ['true','false']
    rental_rates = ['day','month','week','year']
    
    for county in counties:
        for furnish in furnished:
            for rate in rental_rates:
                
                # specify website url 
                url = 'https://www.property24.co.ke/property-to-rent-in-'+str(county)+'-p95?rentalterm='+str(rate)+'&isfurnished='+str(furnish)+'&propertytypes=apartments-flats,townhouses&Page=' + str(page)
                req = requests.get(url).text 
                soup = BeautifulSoup(req,'lxml')
                listings = soup.find_all('span',class_='p24_content')

                for listing in listings:
                    try:
                        property_title = listing.find('span',class_='p24_propertyTitle').text.strip()
                        property_availability = 'For Rent'
                        property_location = listing.find('span',class_='p24_location').text.strip()
                        property_address = listing.find('span',class_='p24_address').text.strip()
                        floor_size = listing.find('span', class_='p24_size', title='Floor Size').text.strip()
                        furnished = furnish
                        rental_rate = rate
                        property_price = listing.find('span',class_='p24_price').text.strip()
                        last_scraped = datetime.now()
                        
                        # Create DataFrame
                        data = pd.DataFrame({
                            'county':[county],
                            'property_title':[property_title],
                            'property_availability':[property_availability],
                            'property_location':[property_location],
                            'property_address':[property_address],
                            'floor_size':[floor_size],
                            'furnished':[furnished],
                            'rental_rate':[rental_rate],
                            'property_price':[property_price],
                            'last_scraped':[last_scraped]
                        })

                        # Append data to bigdata DataFrame
                        bigdata = pd.concat([bigdata,data],ignore_index=True)
                        
                    except Exception as e:
                        pass            

# Handle Database Import Error
table_id = 'project-adrian-julius-aluoch.cronjobs.real_estate_data'
job = client.load_table_from_dataframe(bigdata,table_id)
while job.state != 'DONE':
    time.sleep(4)
    job.reload()
    print(f"Data Upload Status : {job.state}")

### **b. Apartmensts for Sale in Nairobi**

In [ ]:
# Initialize DataFrame outside the loop
bigdata = pd.DataFrame()

for page in trange(1,101):
    counties = ['nairobi']
    
    for county in counties:
        # specify website url 
        url = 'https://www.property24.co.ke/property-for-sale-in-'+str(county)+'-p95?propertytypes=apartments-flats,townhouses&Page=' + str(page)
        req = requests.get(url).text 
        soup = BeautifulSoup(req,'lxml')
        listings = soup.find_all('span',class_='p24_content')

        for listing in listings:
            try:
                property_title = listing.find('span',class_='p24_propertyTitle').text.strip()
                property_availability = 'For Sale'
                property_location = listing.find('span',class_='p24_location').text.strip()
                property_address = listing.find('span',class_='p24_address').text.strip()
                floor_size = listing.find('span',class_='p24_size').text.strip()
                property_price = listing.find('span',class_='p24_price').text.strip()
                rental_rate = np.NAN
                furnished = np.NAN
                last_scraped = datetime.now()
                
                # Create DataFrame
                data = pd.DataFrame({
                    'county':[county],
                    'property_title':[property_title],
                    'property_availability':[property_availability],
                    'property_location':[property_location],
                    'property_address':[property_address],
                    'floor_size':[floor_size],
                    'furnished':[furnished],
                    'rental_rate':[rental_rate],
                    'property_price':[property_price],
                    'last_scraped':[last_scraped]
                })

                # Append data to bigdata DataFrame
                bigdata = pd.concat([bigdata,data],ignore_index=True)
                
            except Exception as e:
                pass   

# Handle Database Import Error
table_id = 'project-adrian-julius-aluoch.cronjobs.real_estate_data'
job = client.load_table_from_dataframe(bigdata,table_id)
while job.state != 'DONE':
    time.sleep(4)
    job.reload()
    print(f"Data Upload Status : {job.state}")

### **c. Apartment for Rent in Mombasa**

In [ ]:
# Initialize DataFrame outside the loop
bigdata = pd.DataFrame()

for page in trange(1,101):
    counties = ['mombasa']
    furnished = ['true','false']
    rental_rates = ['day','month','week','year']
    
    for county in counties:
        for furnish in furnished:
            for rate in rental_rates:
                
                # specify website url 
                url = 'https://www.property24.co.ke/property-to-rent-in-'+str(county)+'-p93?rentalterm='+str(rate)+'&isfurnished='+str(furnish)+'&propertytypes=apartments-flats,townhouses&Page=' + str(page)
                req = requests.get(url).text 
                soup = BeautifulSoup(req,'lxml')
                listings = soup.find_all('span',class_='p24_content')

                for listing in listings:
                    try:
                        property_title = listing.find('span',class_='p24_propertyTitle').text.strip()
                        property_availability = 'For Rent'
                        property_location = listing.find('span',class_='p24_location').text.strip()
                        property_address = listing.find('span',class_='p24_address').text.strip()
                        floor_size = listing.find('span', class_='p24_size', title='Floor Size').text.strip()
                        furnished = furnish
                        rental_rate = rate
                        property_price = listing.find('span',class_='p24_price').text.strip()
                        last_scraped = datetime.now()
                        
                        # Create DataFrame
                        data = pd.DataFrame({
                            'county':[county],
                            'property_title':[property_title],
                            'property_availability':[property_availability],
                            'property_location':[property_location],
                            'property_address':[property_address],
                            'floor_size':[floor_size],
                            'furnished':[furnished],
                            'rental_rate':[rental_rate],
                            'property_price':[property_price],
                            'last_scraped':[last_scraped]
                        })

                        # Append data to bigdata DataFrame
                        bigdata = pd.concat([bigdata,data],ignore_index=True)
                        
                    except Exception as e:
                        pass            

# Handle Database Import Error
table_id = 'project-adrian-julius-aluoch.cronjobs.real_estate_data'
job = client.load_table_from_dataframe(bigdata,table_id)
while job.state != 'DONE':
    time.sleep(4)
    job.reload()
    print(f"Data Upload Status : {job.state}")

### **d. Apartments for Sale in Mombasa**

In [ ]:
# Initialize DataFrame outside the loop
bigdata = pd.DataFrame()

# specify website url 
for page in trange(1,101):
    counties = ['mombasa']
    for county in counties:
        url = 'https://www.property24.co.ke/property-for-sale-in-'+str(county)+'-p93?propertytypes=apartments-flats,townhouses&Page=' + str(page)
        req = requests.get(url).text 
        soup = BeautifulSoup(req,'lxml')

        listings = soup.find_all('span',class_='p24_content')

        for listing in listings:
            try:
                property_title = listing.find('span',class_='p24_propertyTitle').text.strip()
                property_availability = 'For Sale'
                property_location = listing.find('span',class_='p24_location').text.strip()
                property_address = listing.find('span',class_='p24_address').text.strip()
                floor_size = listing.find('span',class_='p24_size').text.strip()
                property_price = listing.find('span',class_='p24_price').text.strip()
                rental_rate = np.NAN
                furnished = np.NAN
                last_scraped = datetime.now()
                data = pd.DataFrame({
                    'county':[county],
                    'property_title':[property_title],
                    'property_availability':[property_availability],
                    'property_location':[property_location],
                    'property_address':[property_address],
                    'floor_size':[floor_size],
                    'furnished':[furnished],
                    'rental_rate':[rental_rate],
                    'property_price':[property_price],
                    'last_scraped':[last_scraped]
                })

                # Append data to bigdata DataFrame
                bigdata = pd.concat([bigdata,data],ignore_index=True)
                
            except Exception as e:
                pass     

# Handle Database Import Error
table_id = 'project-adrian-julius-aluoch.cronjobs.real_estate_data'
job = client.load_table_from_dataframe(bigdata,table_id)
while job.state != 'DONE':
    time.sleep(4)
    job.reload()
    print(f"Data Upload Status : {job.state}")

### **Basic Data Cleaning**

In [ ]:
# Define SQL Query to Retrieve Real Estate Data from Google Cloud BigQuery
sql = (
       'SELECT *'
       'FROM `cronjobs.real_estate_data`'
       )

# Run SQL Query
data = client.query(sql).to_dataframe()
print(f'Rows of Real Estate Data in Google BigQuery : {data.shape[0]:,.0f}\nCols of Real Estate Data in Google BigQuery : {data.shape[1]:,.0f}')

In [ ]:
# Check Total Number of Duplicate Records
duplicated = data.duplicated(subset=[
       'county', 'property_title', 'property_availability',
       'property_location', 'property_address', 'floor_size', 
       'furnished', 'rental_rate', 'property_price'
                                    ]).sum()

# Remove Duplicate Records
data.drop_duplicates(subset=[
       'county', 'property_title', 'property_availability',
       'property_location', 'property_address', 'floor_size', 
       'furnished', 'rental_rate', 'property_price'
                            ],inplace=True)

# Display Initial & Final Number of Duplicate Records
print(f"Initial Shape of Dataset : {data.shape}\nTotal Duplicate Records : {duplicated:,.0f}\nFinal Shape of Dataset : {data.shape}")

In [ ]:
# Drop Original Real Estate Table 
table_id = 'project-adrian-julius-aluoch.cronjobs.real_estate_data'
client.delete_table(table_id)

# Upload Final Real Estate Table
job = client.load_table_from_dataframe(data,table_id)
while job.state != 'DONE':
    time.sleep(1)
    job.reload()
    print(f'Real Estate Data Update : {job.state}')